### EJEMPLO 11: Creamos el fichero u03_util.py donde implementamos varias funciones de dibujo

In [16]:
# FICHERO u03_util.py
# contiene varios métodos que se usarán en varios ejemplos
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, colorConverter
from u03_tools import discrete_scatter, plot_2d_separator, cm2
from sklearn.datasets import make_blobs
from sklearn.svm import LinearSVC

ModuleNotFoundError: No module named 'u03_tools'

In [12]:
cm2 = ListedColormap(['#0000aa', '#ff2020'])
def discrete_scatter(x1, x2, y=None, markers=None, s=10, ax=None, labels=None, padding=.2, alpha=1, c=None, markeredgewidth=None):
    if ax is None:
        ax = plt.gca()
    if y is None:
        y = np.zeros(len(x1))
    unique_y = np.unique(y)
    if markers is None:
        markers = ['o', '^', 'v', 'D', 's', '*', 'p', 'h', 'H', '8', '<', '>'] * 10
    if len(markers) == 1:
        markers = markers * len(unique_y)
    if labels is None:
        labels = unique_y
    lineas = []
    current_cycler = mpl.rcParams['axes.prop_cycle']
    for i, (yy, cycle) in enumerate(zip(unique_y, current_cycler())):
        mask = y == yy
    if c is None:
        color = cycle['color']
    elif len(c) > 1:
        color = c[i]
    else:
        color = c
    # si c es none, usa colores repetidos
    if np.mean(colorConverter.to_rgb(color)) < .4: # frontera clara si marker oscuro
            markeredgecolor = "grey"
    else:
        markeredgecolor = "black"
        lineas.append(ax.plot(x1[mask], x2[mask], markers[i], markersize=s, label=labels[i], alpha=alpha, c=color,markeredgewidth=markeredgewidth, markeredgecolor=markeredgecolor)[0])
    if padding != 0:
        pad1 = x1.std() * padding
        pad2 = x2.std() * padding
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        ax.set_xlim(min(x1.min() - pad1, xlim[0]), max(x1.max() + pad1, xlim[1]))
        ax.set_ylim(min(x2.min() - pad2, ylim[0]), max(x2.max() + pad2, ylim[1]))
    return lineas

In [13]:
def plot_2d_separator(classifier, X, fill=False, ax=None, eps=None, alpha=1, cm=cm2, linewidth=None, threshold=None, linestyle="solid"):
    if eps is None:
        eps = X.std() / 2.
    if ax is None:
        ax = plt.gca()
    x_min, x_max = X[:, 0].min() - eps, X[:, 0].max() + eps
    y_min, y_max = X[:, 1].min() - eps, X[:, 1].max() + eps
    xx = np.linspace(x_min, x_max, 1000)
    yy = np.linspace(y_min, y_max, 1000)
    X1, X2 = np.meshgrid(xx, yy)
    X_grid = np.c_[X1.ravel(), X2.ravel()]

In [14]:
try:
    decision_values = classifier.decision_function(X_grid)
    levels = [0] if threshold is None else [threshold]
    fill_levels = [decision_values.min()] + levels + [decision_values.max()]
except AttributeError:
    # no decision_function
    decision_values = classifier.predict_proba(X_grid)[:, 1]
    levels = [.5] if threshold is None else [threshold]
    fill_levels = [0] + levels + [1]
    if fill:
        ax.contourf(X1, X2, decision_values.reshape(X1.shape), levels=fill_levels, alpha=alpha, cmap=cm)
    else:
        ax.contour(X1, X2, decision_values.reshape(X1.shape), levels=levels, colors="black", alpha=alpha, linewidths=linewidth, linestyles=linestyle, zorder=5)
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        ax.set_xticks(())
        ax.set_yticks(())
def plot_dataset(X, y, axes):
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bs")
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "g^")
    plt.axis(axes)
    plt.grid(True, which='both')
    plt.xlabel(r"$x_1$", fontsize=14)
    plt.ylabel(r"$x_2$", fontsize=14, rotation=0)
def plot_predicciones(clf, axes):
    x0s = np.linspace(axes[0], axes[1], 100)
    x1s = np.linspace(axes[2], axes[3], 100)
    x0, x1 = np.meshgrid(x0s, x1s)
    X = np.c_[x0.ravel(), x1.ravel()]
    y_pred = clf.predict(X).reshape(x0.shape)
    y_decision = clf.decision_function(X).reshape(x0.shape)
    plt.contourf(x0, x1, y_pred, cmap=plt.cm.brg, alpha=0.2)
    plt.contourf(x0, x1, y_decision, cmap=plt.cm.brg, alpha=0.1)

NameError: name 'classifier' is not defined

###  EJEMPLO 12: Generar y dibujar datos linealmente no separables en 2D.


In [15]:
X, y = make_blobs(centers=4, random_state=8)
y = y % 2
discrete_scatter(X[:, 0], X[:, 1], y)
plt.xlabel("Característica 0")
plt.ylabel("Característica 1")
plt.plot()

NameError: name 'make_blobs' is not defined

### EJEMPLO 13: Crear y entrenar un modelo clasificador SVM para los datos del EJEMPLO 12.

In [ ]:
lineal_svm = LinearSVC().fit(X, y)
plot_2d_separator(lineal_svm, X)
discrete_scatter(X[:, 0], X[:, 1], y)
plt.xlabel("Característica 0")
plt.ylabel("Característica 1")

### EJEMPLO 14: Crear y entrenar un modelo clasificador SVM para los datos del EJEMPLO 12.

In [ ]:
X_new = np.hstack( [X, X[:, 1:] ** 2] )
ax = plt.figure(figsize=(12,11)).add_subplot(projection='3d')
mascara = y == 0
ax.scatter(X_new[mascara, 0], X_new[mascara, 1], X_new[mascara, 2], c='b', cmap= cm2,
       s=60, edgecolor='k')
ax.scatter(X_new[~mascara, 0], X_new[~mascara, 1], X_new[~mascara, 2], c='r', marker='^',
       cmap=cm2, s=60, edgecolor='k')
ax.set_xlabel("característica0")
ax.set_ylabel("característica1")
ax.set_zlabel("caracerística1 ** 2")
ax.view_init(elev=-152., azim=-26, roll=0) # plt.show()

 ###  EJEMPLO 15: Dibujar el plano 3D que separa los datos de las dos clases.


In [ ]:
lineal_svm_3d = LinearSVC().fit(X_new, y)
coef, intercept = lineal_svm_3d.coef_.ravel(), lineal_svm_3d.intercept_
# Mostrar la frontera de decisión
ax = plt.figure(figsize=(10,11)).add_subplot(projection='3d')
xx = np.linspace(X_new[:, 0].min() - 2, X_new[:, 0].max() + 2, 50)
yy = np.linspace(X_new[:, 1].min() - 2, X_new[:, 1].max() + 2, 50)
XX, YY = np.meshgrid(xx, yy)
ZZ = (coef[0] * XX + coef[1] * YY + intercept) / -coef[2]
ax.plot_surface(XX, YY, ZZ, rstride=8, cstride=8, alpha=0.3)
ax.scatter(X_new[mascara, 0], X_new[mascara, 1], X_new[mascara, 2], c='b',
       cmap=cm2, s=60, edgecolor='k')
ax.scatter(X_new[~mascara, 0], X_new[~mascara, 1], X_new[~mascara, 2], c='r',
       marker='^', cmap=cm2, s=60, edgecolor='k')
ax.set_xlabel("característica0")
ax.set_ylabel("característica1")
ax.set_zlabel("caracerística1 ** 2")
ax.view_init(elev=-156., azim=-20, roll=0) # plt.show()